In [4]:
pip install jenkspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.9/650.9 kB 927.1 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import jenkspy
import os

In [7]:
urbanization_df = pd.read_csv('API_SP.URB.TOTL.IN.ZS_DS2_en_csv_v2_19523.csv')  # Urbanization data
# keep Country Code and 2010–2022 years
years = [str(year) for year in range(2010, 2023)]
urbanization_trimmed_df = urbanization_df[['Country Code'] + years]

# 1. RMSE under SSP-Moderate Speed

In [172]:
# 1. Read your predictions (clustering-based approach)
# HC
#cluster_SSP1_df = pd.read_excel("urban_pred_2010_2022_SSP1.xlsx", index_col=0)
#DTW
cluster_SSP1_df = pd.read_excel("DTW_urban_proj_2010-2022_WB/DTW_urban_pred_2010_2022_middle.xlsx", index_col=0)
#PCAF
#cluster_SSP1_df = pd.read_excel("PCAF_urban_pred_2010_2022/PCAF_urban_pred_2010_2022_middle.xlsx", index_col=0)

# 2. Read the prediction of Chen et al.
chen_SSP1_df = pd.read_excel("WB/SSP1_WB.xls", index_col=0)

# 3. Reading Jiang & O'Neill's predictions and filtering SSP1 scenarios
jiang_raw_df = pd.read_excel("urbproj_all.xlsx", sheet_name="data")
jiang_SSP2_df = jiang_raw_df[jiang_raw_df['Scenario'] == 'SSP2_v9_131230']

In [174]:
years = [str(y) for y in range(2011, 2023)]
cluster_SSP1_df = cluster_SSP1_df[cluster_SSP1_df.columns.intersection(years)]
chen_SSP1_df = chen_SSP1_df[[year for year in range(2011, 2023)]]
# Add Country columns to cluster_SSP1_df
cluster_SSP1_df = cluster_SSP1_df.reset_index().rename(columns={'index': 'Country'})
# Add Country columns to chen_SSP1_df
chen_SSP1_df = chen_SSP1_df.reset_index().rename(columns={'index': 'Country'})
jiang_SSP2_df = jiang_SSP2_df[['Region', 2010, 2015, 2020]]
# Add Country columns to jiang_SSP2_df
jiang_SSP2_df = jiang_SSP2_df.rename(columns={'Region': 'Country'})
cluster_SSP1_df = cluster_SSP1_df.rename(columns={'Country Code': 'Country'})
chen_SSP1_df = chen_SSP1_df.rename(columns={'country or area': 'Country'})

## 1.1 Define the RMSE calculation function

In [14]:
def calculate_rmse_debug(true_df, pred_df, year_range):
    true_df = true_df.set_index('Country Code')
    pred_df = pred_df.set_index('Country')

    common_countries = true_df.index.intersection(pred_df.index)
    common_years = [year for year in year_range if year in true_df.columns and year in pred_df.columns]

    print("Common countries count:", len(common_countries))
    print("Common years:", common_years)

    rmses = {}
    for country in common_countries:
        true_vals = true_df.loc[country, common_years]
        pred_vals = pred_df.loc[country, common_years]

        if true_vals.isnull().any() or pred_vals.isnull().any():
            print(f"[SKIPPED] Missing data for {country}")
            continue

        rmse = np.sqrt(np.mean((true_vals.values - pred_vals.values) ** 2))
        rmses[country] = rmse

    return rmses

In [185]:
rmse_cluster_SSP1 = calculate_rmse_debug(urbanization_trimmed_df.copy(), cluster_SSP1_df.copy(), [str(y) for y in range(2011, 2023)])
# Converting a Dictionary to a DataFrame
rmse_cluster_SSP1_df = pd.DataFrame(list(rmse_cluster_SSP1.items()), columns=["Country Code", "RMSE"])

Common countries count: 165
Common years: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [186]:
rmse_cluster_SSP1_df

,Country Code,RMSE
0,AFG,1.680882
1,AGO,0.388919
2,ALB,4.763402
3,ARE,1.073784
4,ARG,0.461400
...,...,...
160,VUT,2.463731
161,YEM,0.220211
162,ZAF,0.524830
163,ZMB,0.053437


In [187]:
# Standardized listings
urbanization_trimmed_df.columns = urbanization_trimmed_df.columns.astype(str).str.strip()
chen_SSP1_df.columns = chen_SSP1_df.columns.astype(str).str.strip()
jiang_SSP2_df.columns = jiang_SSP2_df.columns.astype(str).str.strip()

In [188]:
rmse_chen_SSP1 = calculate_rmse_debug(urbanization_trimmed_df.copy(), chen_SSP1_df.copy(), [str(y) for y in range(2011, 2023)])
rmse_chen_SSP1_df = pd.DataFrame(list(rmse_chen_SSP1.items()), columns=["Country Code", "RMSE"])

Common countries count: 188
Common years: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [189]:
rmse_chen_SSP1_df

,Country Code,RMSE
0,AFG,2.208677
1,AGO,0.601417
2,ALB,3.812187
3,AND,2.600285
4,ARE,1.329793
...,...,...
183,VUT,3.151465
184,YEM,0.082062
185,ZAF,0.382778
186,ZMB,0.121950


In [190]:
rmse_jiang_SSP2 = calculate_rmse_debug(urbanization_trimmed_df.copy(), jiang_SSP2_df.copy(), ['2010', '2015', '2020'])
rmse_jiang_SSP2_df = pd.DataFrame(list(rmse_jiang_SSP2.items()), columns=["Country Code", "RMSE"])

Common countries count: 188
Common years: ['2010', '2015', '2020']


In [191]:
rmse_jiang_SSP2_df

,Country Code,RMSE
0,ABW,7.329215
1,AFG,1.118086
2,AGO,1.275545
3,ALB,1.683420
4,ARE,0.337559
...,...,...
183,WSM,4.219903
184,YEM,0.458328
185,ZAF,0.351303
186,ZMB,3.212468


In [192]:
# Result Saving 
# HC Result
#rmse_cluster_SSP1_df.to_csv('RMSE/SSP_Moderate/rmse_hierarchicalcluster_SSP1.csv', index=False)
# DTW Result
rmse_cluster_SSP1_df.to_csv('RMSE/SSP_Moderate/rmse_DTW_SSP1.csv', index=False)
# PCAF Result
#rmse_cluster_SSP1_df.to_csv('RMSE/SSP_Moderate/rmse_PCAF_SSP1.csv', index=False)
#rmse_chen_SSP1_df.to_csv('RMSE/SSP_Moderate/rmse_chen_SSP1.csv', index=False)
#rmse_jiang_SSP2_df.to_csv('RMSE/SSP_Moderate/rmse_jiang_SSP2.csv', index=False)

# 2.RMSE under SSP-Slow Speed

In [193]:
# Read predictions (clustering-based approach)
# HC
#cluster_SSP2_df = pd.read_excel("urban_pred_2010_2022_SSP2.xlsx", index_col=0)
#DTW 
cluster_SSP2_df = pd.read_excel("DTW_urban_proj_2010-2022_WB/DTW_urban_pred_2010_2022_low.xlsx", index_col=0)
#PCAF 
#cluster_SSP2_df = pd.read_excel("PCAF_urban_pred_2010_2022/PCAF_urban_pred_2010_2022_low.xlsx", index_col=0)
# Read the predictions of Chen et al.
chen_SSP2_df = pd.read_excel("WB/SSP2_WB.xls", index_col=0)
# Reading Jiang & O'Neill's predictions to filter SSP3 scenarios
jiang_raw_df = pd.read_excel("urbproj_all.xlsx", sheet_name="data")
jiang_SSP3_df = jiang_raw_df[jiang_raw_df['Scenario'] == 'SSP3_v9_131230']

In [194]:
# selecting years and Adding Country columns
years = [str(y) for y in range(2011, 2023)]
cluster_SSP2_df = cluster_SSP2_df[cluster_SSP2_df.columns.intersection(years)]
chen_SSP2_df = chen_SSP2_df[[year for year in range(2011, 2023)]]
cluster_SSP2_df = cluster_SSP2_df.reset_index().rename(columns={'index': 'Country'})
chen_SSP2_df = chen_SSP2_df.reset_index().rename(columns={'index': 'Country'})
jiang_SSP3_df = jiang_SSP3_df[['Region', 2010, 2015, 2020]]
jiang_SSP3_df = jiang_SSP3_df.rename(columns={'Region': 'Country'})
cluster_SSP2_df = cluster_SSP2_df.rename(columns={'Country Code': 'Country'})
chen_SSP2_df = chen_SSP2_df.rename(columns={'country or area': 'Country'})

In [195]:
rmse_cluster_SSP2 = calculate_rmse_debug(urbanization_trimmed_df.copy(), cluster_SSP2_df.copy(), [str(y) for y in range(2011, 2023)])
# Converting a Dictionary to a DataFrame
rmse_cluster_SSP2_df = pd.DataFrame(list(rmse_cluster_SSP2.items()), columns=["Country Code", "RMSE"])

Common countries count: 165
Common years: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [197]:
# Standardized columns
urbanization_trimmed_df.columns = urbanization_trimmed_df.columns.astype(str).str.strip()
chen_SSP2_df.columns = chen_SSP2_df.columns.astype(str).str.strip()
jiang_SSP3_df.columns = jiang_SSP3_df.columns.astype(str).str.strip()

In [198]:
rmse_chen_SSP2 = calculate_rmse_debug(urbanization_trimmed_df.copy(), chen_SSP2_df.copy(), [str(y) for y in range(2011, 2023)])
rmse_chen_SSP2_df = pd.DataFrame(list(rmse_chen_SSP2.items()), columns=["Country Code", "RMSE"])

Common countries count: 188
Common years: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [200]:
rmse_jiang_SSP3 = calculate_rmse_debug(urbanization_trimmed_df.copy(), jiang_SSP3_df.copy(), ['2010', '2015', '2020'])
rmse_jiang_SSP3_df = pd.DataFrame(list(rmse_jiang_SSP3.items()), columns=["Country Code", "RMSE"])

Common countries count: 188
Common years: ['2010', '2015', '2020']


In [202]:
# Result Saving 
#HC Result
#rmse_cluster_SSP2_df.to_csv('RMSE/SSP_Slow/rmse_hierarchicalcluster_SSP2.csv', index=False)
#DTW Result
rmse_cluster_SSP1_df.to_csv('RMSE/SSP_Slow/rmse_DTW_SSP2.csv', index=False)
#PCAF Result
#rmse_cluster_SSP2_df.to_csv('RMSE/SSP_Slow/rmse_PCAF_SSP2.csv', index=False)
#rmse_chen_SSP2_df.to_csv('RMSE/SSP_Slow/rmse_chen_SSP2.csv', index=False)
#rmse_jiang_SSP3_df.to_csv('RMSE/SSP_Slow/rmse_jiang_SSP3.csv', index=False)

# 3.RMSE under SSP-fast Speed

In [203]:
# Read predictions (clustering-based approach)
#HC
#cluster_SSP5_df = pd.read_excel("urban_pred_2010_2022_SSP5.xlsx", index_col=0)
#DTW
cluster_SSP5_df = pd.read_excel("DTW_urban_proj_2010-2022_WB/DTW_urban_pred_2010_2022_low.xlsx", index_col=0)
#PCAF
#cluster_SSP5_df = pd.read_excel("PCAF_urban_pred_2010_2022/PCAF_urban_pred_2010_2022_fast.xlsx", index_col=0)
# Read the predictions of Chen et al.
chen_SSP5_df = pd.read_excel("WB/SSP5_WB.xls", index_col=0)
# 读Taking Jiang & O'Neill's predictions and screening SSP1 scenarios
jiang_raw_df = pd.read_excel("urbproj_all.xlsx", sheet_name="data")
jiang_SSP1_df = jiang_raw_df[jiang_raw_df['Scenario'] == 'SSP1_v9_131230']

In [204]:
# selecting years and Adding Country columns
years = [str(y) for y in range(2011, 2023)]
cluster_SSP5_df = cluster_SSP5_df[cluster_SSP5_df.columns.intersection(years)]
chen_SSP5_df = chen_SSP5_df[[year for year in range(2011, 2023)]]
cluster_SSP5_df = cluster_SSP5_df.reset_index().rename(columns={'index': 'Country'})
chen_SSP5_df = chen_SSP5_df.reset_index().rename(columns={'index': 'Country'})
jiang_SSP1_df = jiang_SSP1_df[['Region', 2010, 2015, 2020]]
jiang_SSP1_df = jiang_SSP1_df.rename(columns={'Region': 'Country'})
cluster_SSP5_df = cluster_SSP5_df.rename(columns={'Country Code': 'Country'})
chen_SSP5_df = chen_SSP5_df.rename(columns={'country or area': 'Country'})

In [205]:
rmse_cluster_SSP5 = calculate_rmse_debug(urbanization_trimmed_df.copy(), cluster_SSP5_df.copy(), [str(y) for y in range(2011, 2023)])
# Converting a Dictionary to a DataFrame
rmse_cluster_SSP5_df = pd.DataFrame(list(rmse_cluster_SSP5.items()), columns=["Country Code", "RMSE"])

Common countries count: 165
Common years: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [207]:
# Standardized columns
urbanization_trimmed_df.columns = urbanization_trimmed_df.columns.astype(str).str.strip()
chen_SSP5_df.columns = chen_SSP5_df.columns.astype(str).str.strip()
jiang_SSP1_df.columns = jiang_SSP1_df.columns.astype(str).str.strip()

In [208]:
rmse_chen_SSP5 = calculate_rmse_debug(urbanization_trimmed_df.copy(), chen_SSP5_df.copy(), [str(y) for y in range(2011, 2023)])
rmse_chen_SSP5_df = pd.DataFrame(list(rmse_chen_SSP5.items()), columns=["Country Code", "RMSE"])

Common countries count: 188
Common years: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [210]:
rmse_jiang_SSP1 = calculate_rmse_debug(urbanization_trimmed_df.copy(), jiang_SSP1_df.copy(), ['2010', '2015', '2020'])
rmse_jiang_SSP1_df = pd.DataFrame(list(rmse_jiang_SSP1.items()), columns=["Country Code", "RMSE"])

Common countries count: 188
Common years: ['2010', '2015', '2020']


In [212]:
# Result Saving 
#HC Result
#rmse_cluster_SSP5_df.to_csv('RMSE/SSP_Fast/rmse_hierarchicalcluster_SSP5.csv', index=False)
#DTW Result
rmse_cluster_SSP1_df.to_csv('RMSE/SSP_Fast/rmse_DTW_SSP5.csv', index=False)
#PCAF Result
#rmse_cluster_SSP5_df.to_csv('RMSE/SSP_Fast/rmse_PCAF_SSP5.csv', index=False)
#rmse_chen_SSP5_df.to_csv('RMSE/SSP_Fast/rmse_chen_SSP5.csv', index=False)
#rmse_jiang_SSP1_df.to_csv('RMSE/SSP_Fast/rmse_jiang_SSP1.csv', index=False)

# 4.RMSE under SSP-4

In [38]:
# Read predictions (clustering-based approach)
#HC
#cluster_SSP4_df = pd.read_excel("urban_proj_2010-2022_WB/HC_urban_pred_2010_2022_ssp4.xlsx", index_col=0)
#DTW
#cluster_SSP4_df = pd.read_excel("DTW_urban_proj_2010-2022_WB/DTW_urban_pred_2010_2022_ssp4.xlsx", index_col=0)
#PCAF
cluster_SSP4_df = pd.read_excel("PCAF_urban_pred_2010_2022/PCAF_urban_pred_2010_2022_ssp4.xlsx", index_col=0)
# Read the predictions of Chen et al.
chen_SSP4_df = pd.read_excel("WB/SSP4_WB.xls", index_col=0)

# 读Taking Jiang & O'Neill's predictions and screening SSP4 scenarios
jiang_raw_df = pd.read_excel("urbproj_all.xlsx", sheet_name="data")
jiang_SSP4_df = jiang_raw_df[jiang_raw_df['Scenario'] == 'SSP4_v9_131230']

In [39]:
# selecting years and Adding Country columns
years = [str(y) for y in range(2011, 2023)]
cluster_SSP4_df = cluster_SSP4_df[cluster_SSP4_df.columns.intersection(years)]
chen_SSP4_df = chen_SSP4_df[[year for year in range(2011, 2023)]]
cluster_SSP4_df = cluster_SSP4_df.reset_index().rename(columns={'index': 'Country'})
chen_SSP4_df = chen_SSP4_df.reset_index().rename(columns={'index': 'Country'})
jiang_SSP4_df = jiang_SSP4_df[['Region', 2010, 2015, 2020]]
jiang_SSP4_df = jiang_SSP4_df.rename(columns={'Region': 'Country'})
cluster_SSP4_df = cluster_SSP4_df.rename(columns={'Country Code': 'Country'})
chen_SSP4_df = chen_SSP4_df.rename(columns={'country or area': 'Country'})

In [40]:
rmse_cluster_SSP4 = calculate_rmse_debug(urbanization_trimmed_df.copy(), cluster_SSP4_df.copy(), [str(y) for y in range(2011, 2023)])
# Converting a Dictionary to a DataFrame
rmse_cluster_SSP4_df = pd.DataFrame(list(rmse_cluster_SSP4.items()), columns=["Country Code", "RMSE"])

Common countries count: 165
Common years: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [41]:
rmse_cluster_SSP4_df

,Country Code,RMSE
0,AFG,1.561466
1,AGO,0.424290
2,ALB,4.080557
3,ARE,1.318840
4,ARG,0.824758
...,...,...
160,VUT,3.817448
161,YEM,1.173320
162,ZAF,1.064646
163,ZMB,0.148435


In [42]:
# Standardized columns
urbanization_trimmed_df.columns = urbanization_trimmed_df.columns.astype(str).str.strip()
chen_SSP4_df.columns = chen_SSP4_df.columns.astype(str).str.strip()
jiang_SSP4_df.columns = jiang_SSP4_df.columns.astype(str).str.strip()

In [43]:
rmse_chen_SSP4 = calculate_rmse_debug(urbanization_trimmed_df.copy(), chen_SSP4_df.copy(), [str(y) for y in range(2011, 2023)])
rmse_chen_SSP4_df = pd.DataFrame(list(rmse_chen_SSP4.items()), columns=["Country Code", "RMSE"])

Common countries count: 188
Common years: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022']


In [44]:
rmse_chen_SSP4_df

,Country Code,RMSE
0,AFG,0.783042
1,AGO,0.601417
2,ALB,3.812187
3,AND,2.600285
4,ARE,1.329793
...,...,...
183,VUT,3.151465
184,YEM,1.477518
185,ZAF,0.382778
186,ZMB,0.121950


In [45]:
rmse_jiang_SSP4 = calculate_rmse_debug(urbanization_trimmed_df.copy(), jiang_SSP4_df.copy(), ['2010', '2015', '2020'])
rmse_jiang_SSP4_df = pd.DataFrame(list(rmse_jiang_SSP4.items()), columns=["Country Code", "RMSE"])

Common countries count: 188
Common years: ['2010', '2015', '2020']


In [46]:
rmse_jiang_SSP4_df

,Country Code,RMSE
0,ABW,7.329215
1,AFG,3.343213
2,AGO,0.950007
3,ALB,0.609694
4,ARE,0.337559
...,...,...
183,WSM,7.755957
184,YEM,2.822979
185,ZAF,0.951245
186,ZMB,2.308271


In [47]:
# Result Saving 
#HC Result
#rmse_cluster_SSP4_df.to_csv('RMSE/SSP_4/rmse_hierarchicalcluster_SSP4.csv', index=False)
#DTW Result
#rmse_cluster_SSP4_df.to_csv('RMSE/SSP_4/rmse_DTW_SSP4.csv', index=False)
#PCAF Result
rmse_cluster_SSP4_df.to_csv('RMSE/SSP_4/rmse_PCAF_SSP4.csv', index=False)
#rmse_chen_SSP4_df.to_csv('RMSE/SSP_4/rmse_chen_SSP4.csv', index=False)
#rmse_jiang_SSP4_df.to_csv('RMSE/SSP_4/rmse_jiang_SSP4.csv', index=False)

# 5.Comparing the results of RMSE

## 5.1 Comparison under SSP-Fast Speed

In [222]:
folder_path = "RMSE/SSP_Fast"

# Search for CSV files
files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Initializing the results container
summary_dict = {}

# Analyze each file
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)

    if 'RMSE' not in df.columns:
        print(f"[跳过] 文件 {file} 中不包含 RMSE 列")
        continue

    rmse_values = df['RMSE'].dropna().values


# Calling jenkspy
    breaks = jenkspy.jenks_breaks(rmse_values, 5)

    # Counting the number of countries in each subparagraph
    bin_labels = []
    bin_counts = []

    for i in range(5):
        lower = breaks[i]
        upper = breaks[i+1]
        count = ((rmse_values > lower) & (rmse_values <= upper)).sum() if i != 0 else ((rmse_values >= lower) & (rmse_values <= upper)).sum()
        bin_labels.append(f"{lower:.2f}–{upper:.2f}")
        bin_counts.append(count)

    summary_dict[file] = {
        "RMSE Range": bin_labels,
        "Number of Countries": bin_counts
    }

# Show results
for file, summary in summary_dict.items():
    print(f"\n📊 RMSE 分布 - {file}")
    display(pd.DataFrame(summary))


📊 RMSE 分布 - rmse_chen_SSP5.csv


,RMSE Range,Number of Countries
0,0.05–0.86,37
1,0.86–1.82,41
2,1.82–2.97,48
3,2.97–4.60,46
4,4.60–6.70,16



📊 RMSE 分布 - rmse_DTW_SSP5.csv


,RMSE Range,Number of Countries
0,0.02–0.63,54
1,0.63–1.28,49
2,1.28–2.14,33
3,2.14–3.40,17
4,3.40–5.63,12



📊 RMSE 分布 - rmse_hierarchicalcluster_SSP5.csv


,RMSE Range,Number of Countries
0,0.08–1.12,62
1,1.12–2.22,55
2,2.22–3.83,40
3,3.83–5.56,7
4,5.56–8.22,1



📊 RMSE 分布 - rmse_jiang_SSP1.csv


,RMSE Range,Number of Countries
0,0.00–4.11,108
1,4.11–8.22,57
2,8.22–12.93,18
3,12.93–25.32,4
4,25.32–36.50,1



📊 RMSE 分布 - rmse_PCAF_SSP5.csv


,RMSE Range,Number of Countries
0,0.04–0.70,41
1,0.70–1.49,40
2,1.49–2.35,40
3,2.35–3.55,29
4,3.55–6.03,15


## 5.2 Comparison under SSP-Moderate Speed

In [223]:
folder_path = "RMSE/SSP_Moderate"

# Search for CSV files
files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Initializing the results container
summary_dict = {}

# Analyze each file
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)

    if 'RMSE' not in df.columns:
        print(f"[跳过] 文件 {file} 中不包含 RMSE 列")
        continue

    rmse_values = df['RMSE'].dropna().values


# Calling jenkspy
    breaks = jenkspy.jenks_breaks(rmse_values, 5)

    # Counting the number of countries in each subparagraph
    bin_labels = []
    bin_counts = []

    for i in range(5):
        lower = breaks[i]
        upper = breaks[i+1]
        count = ((rmse_values > lower) & (rmse_values <= upper)).sum() if i != 0 else ((rmse_values >= lower) & (rmse_values <= upper)).sum()
        bin_labels.append(f"{lower:.2f}–{upper:.2f}")
        bin_counts.append(count)

    summary_dict[file] = {
        "RMSE Range": bin_labels,
        "Number of Countries": bin_counts
    }

# Show results
for file, summary in summary_dict.items():
    print(f"\n📊 RMSE 分布 - {file}")
    display(pd.DataFrame(summary))


📊 RMSE 分布 - rmse_chen_SSP1.csv


,RMSE Range,Number of Countries
0,0.07–0.64,47
1,0.64–1.37,47
2,1.37–2.29,40
3,2.29–3.53,32
4,3.53–5.40,22



📊 RMSE 分布 - rmse_DTW_SSP1.csv


,RMSE Range,Number of Countries
0,0.02–0.63,54
1,0.63–1.28,49
2,1.28–2.14,33
3,2.14–3.40,17
4,3.40–5.63,12



📊 RMSE 分布 - rmse_hierarchicalcluster_SSP1.csv


,RMSE Range,Number of Countries
0,0.01–0.54,44
1,0.54–1.15,55
2,1.15–1.96,35
3,1.96–3.17,20
4,3.17–5.87,11



📊 RMSE 分布 - rmse_jiang_SSP2.csv


,RMSE Range,Number of Countries
0,0.00–3.29,113
1,3.29–7.97,56
2,7.97–16.70,16
3,16.70–27.15,2
4,27.15–37.70,1



📊 RMSE 分布 - rmse_PCAF_SSP1.csv


,RMSE Range,Number of Countries
0,0.00–0.76,60
1,0.76–1.49,55
2,1.49–2.46,28
3,2.46–3.79,16
4,3.79–6.05,6


## 5.3 Comparison under SSP-Slow Speed

In [224]:
folder_path = "RMSE/SSP_Slow"

# Search for CSV files
files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Initializing the results container
summary_dict = {}

# Analyze each file
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)

    if 'RMSE' not in df.columns:
        print(f"[跳过] 文件 {file} 中不包含 RMSE 列")
        continue

    rmse_values = df['RMSE'].dropna().values


# Calling jenkspy
    breaks = jenkspy.jenks_breaks(rmse_values, 5)

    # Counting the number of countries in each subparagraph
    bin_labels = []
    bin_counts = []

    for i in range(5):
        lower = breaks[i]
        upper = breaks[i+1]
        count = ((rmse_values > lower) & (rmse_values <= upper)).sum() if i != 0 else ((rmse_values >= lower) & (rmse_values <= upper)).sum()
        bin_labels.append(f"{lower:.2f}–{upper:.2f}")
        bin_counts.append(count)

    summary_dict[file] = {
        "RMSE Range": bin_labels,
        "Number of Countries": bin_counts
    }

# Show results
for file, summary in summary_dict.items():
    print(f"\n📊 RMSE 分布 - {file}")
    display(pd.DataFrame(summary))


📊 RMSE 分布 - rmse_chen_SSP2.csv


,RMSE Range,Number of Countries
0,0.06–0.80,67
1,0.80–1.51,47
2,1.51–2.63,42
3,2.63–4.01,25
4,4.01–6.33,7



📊 RMSE 分布 - rmse_DTW_SSP2.csv


,RMSE Range,Number of Countries
0,0.02–0.63,54
1,0.63–1.28,49
2,1.28–2.14,33
3,2.14–3.40,17
4,3.40–5.63,12



📊 RMSE 分布 - rmse_hierarchicalcluster_SSP2.csv


,RMSE Range,Number of Countries
0,0.06–0.70,66
1,0.70–1.39,38
2,1.39–2.19,36
3,2.19–3.37,19
4,3.37–6.36,6



📊 RMSE 分布 - rmse_jiang_SSP3.csv


,RMSE Range,Number of Countries
0,0.00–3.13,115
1,3.13–7.42,52
2,7.42–15.79,18
3,15.79–28.80,2
4,28.80–38.32,1



📊 RMSE 分布 - rmse_PCAF_SSP2.csv


,RMSE Range,Number of Countries
0,0.02–0.59,53
1,0.59–1.29,43
2,1.29–2.21,42
3,2.21–4.10,22
4,4.10–6.79,5


## 5.4 Comparison under SSP-4 Speed

In [48]:
folder_path = "RMSE/SSP_4"

# Search for CSV files
files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]

# Initializing the results container
summary_dict = {}

# Analyze each file
for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)

    if 'RMSE' not in df.columns:
        print(f"[跳过] 文件 {file} 中不包含 RMSE 列")
        continue

    rmse_values = df['RMSE'].dropna().values


# Calling jenkspy
    breaks = jenkspy.jenks_breaks(rmse_values, 5)

    # Counting the number of countries in each subparagraph
    bin_labels = []
    bin_counts = []

    for i in range(5):
        lower = breaks[i]
        upper = breaks[i+1]
        count = ((rmse_values > lower) & (rmse_values <= upper)).sum() if i != 0 else ((rmse_values >= lower) & (rmse_values <= upper)).sum()
        bin_labels.append(f"{lower:.2f}–{upper:.2f}")
        bin_counts.append(count)

    summary_dict[file] = {
        "RMSE Range": bin_labels,
        "Number of Countries": bin_counts
    }

# Show results
for file, summary in summary_dict.items():
    print(f"\n📊 RMSE 分布 - {file}")
    display(pd.DataFrame(summary))


📊 RMSE 分布 - rmse_chen_SSP4.csv


,RMSE Range,Number of Countries
0,0.07–0.94,71
1,0.94–1.95,52
2,1.95–3.19,39
3,3.19–4.42,19
4,4.42–5.40,7



📊 RMSE 分布 - rmse_DTW_SSP4.csv


,RMSE Range,Number of Countries
0,0.02–0.89,63
1,0.89–1.73,34
2,1.73–2.81,40
3,2.81–4.23,19
4,4.23–6.01,9



📊 RMSE 分布 - rmse_hierarchicalcluster_SSP4.csv


,RMSE Range,Number of Countries
0,0.01–0.70,47
1,0.70–1.41,37
2,1.41–2.37,38
3,2.37–3.83,34
4,3.83–5.87,9



📊 RMSE 分布 - rmse_jiang_SSP4.csv


,RMSE Range,Number of Countries
0,0.00–2.37,69
1,2.37–5.10,56
2,5.10–9.31,46
3,9.31–16.70,14
4,16.70–37.70,3



📊 RMSE 分布 - rmse_PCAF_SSP4.csv


,RMSE Range,Number of Countries
0,0.00–0.74,45
1,0.74–1.70,50
2,1.70–2.68,40
3,2.68–3.82,16
4,3.82–6.03,14
